In [2]:
pip install google-generativeai flask pillow requests


In [3]:
import os
import google.generativeai as genai
from flask import Flask, request, jsonify
from PIL import Image
import io
import requests

# Initialize Flask app
app = Flask(__name__)

# Configure API keys
os.environ['GOOGLE_API_KEY'] = 'YOUR_GOOGLE_API_KEY'
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

# Initialize Gemini model
gemini_model = genai.GenerativeModel('gemini-2.5')

# Initialize PaLM model
palm_model = genai.GenerativeModel('palm-2')

@app.route('/chat', methods=['POST'])
def chat():
    user_input = request.json.get('message')
    image_url = request.json.get('image_url')

    # Process text input
    if user_input:
        response = gemini_model.generate_content(user_input)
        return jsonify({'response': response.text})

    # Process image input
    if image_url:
        image_data = requests.get(image_url).content
        image = Image.open(io.BytesIO(image_data))
        # Process image with PaLM or Gemini (implement image processing logic here)
        return jsonify({'response': 'Image processed successfully'})

    return jsonify({'error': 'No valid input provided'}), 400

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (inotify)


In [8]:
from IPython.display import display, HTML

html_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Multimodal Chatbot</title>
</head>
<body>
    <h1>Multimodal Chatbot</h1>
    <textarea id="message" placeholder="Type your message here..."></textarea><br>
    <input type="file" id="image" accept="image/*"><br>
    <button onclick="sendMessage()">Send</button>

    <h2>Response:</h2>
    <p id="response"></p>

    <script>
        async function sendMessage() {
            const message = document.getElementById('message').value;
            const image = document.getElementById('image').files[0];
            const formData = new FormData();
            formData.append('message', message);
            if (image) {
                formData.append('image', image);
            }

            const response = await fetch('/chat', {
                method: 'POST',
                body: formData
            });
            const data = await response.json();
            document.getElementById('response').innerText = data.response || data.error;
        }
    </script>
</body>
</html>
"""

display(HTML(html_content))